# 3.4 Data Sharing
Science is much more impactful once it's shared. Therefore, we are going to learn how to 
open up our scientific output from a cloud platform, so that is openly available - and 
has the chance to make the impact it should.
- Load data
- ...


Set configurations

In [1]:
import openeo
from sentinelhub import (SHConfig,
                         SentinelHubRequest,
                         DataCollection,
                         MimeType, CRS,
                         BBox, bbox_to_dimensions,
                         geometry)
import numpy as np
import leafmap

import eo_utils

In [2]:
config = SHConfig()
config.sh_client_id = %env SH_CLIENT_ID
config.sh_client_secret = %env SH_CLIENT_SECRET

In [3]:
conn = openeo.connect('https://jjdxlu8vwl.execute-api.eu-central-1.amazonaws.com/production')
# conn = openeo.connect('https://openeo-dev.sinergise.com/testing/')

In [4]:
conn.authenticate_basic(username=config.sh_client_id, password=config.sh_client_secret)

<Connection to 'https://jjdxlu8vwl.execute-api.eu-central-1.amazonaws.com/production/' with BasicBearerAuth>

In [5]:
# Use this for more 
# https://github.com/openEOPlatform/sample-notebooks/blob/main/openEO%20Platfrom%20-%20Basics.ipynb
# https://github.com/Open-EO/openeo-community-examples/tree/main/python

In [6]:
# conn.list_processes()

### Select Area of Interest
- select a point of interest

In [7]:
import math

# Function to create a bounding box around a point
def create_bounding_box(latitude, longitude, distance_km):
    # Radius of the Earth in kilometers
    earth_radius_km = 6371

    # Convert latitude and longitude from degrees to radians
    lat_rad = math.radians(latitude)
    lon_rad = math.radians(longitude)

    # Calculate the angular distance covered by the given distance_km
    angular_distance = distance_km / earth_radius_km

    # Calculate the latitude and longitude offsets
    lat_offset = math.degrees(angular_distance)
    lon_offset = math.degrees(angular_distance / math.cos(lat_rad))

    # Calculate the coordinates of the southwest and northeast corners
    sw_lat = latitude - lat_offset
    sw_lon = longitude - lon_offset
    ne_lat = latitude + lat_offset
    ne_lon = longitude + lon_offset

    # Return the bounding box as a tuple (sw_lat, sw_lon, ne_lat, ne_lon)
    return (sw_lat, sw_lon, ne_lat, ne_lon)

In [25]:
# import pyproj
# def reproject_lat_long(long, lat, dest_epsg=32632):
#     source_proj = pyproj.Proj("epsg:4326")
#     dest_proj = pyproj.Proj(f"epsg:{dest_epsg}")
#     x, y = pyproj.transform(source_proj, dest_proj, long, lat)
#     return x, y

# reproject_lat_long(x_targ_two, y_targ_one)
#point_coords()
# get center point, 
# convert to UTM 
# expand to 100 by 100 pixel 
# convert back to long lat
# generate bbox from them 
# and use in the process graph
# center = [46.497012, 11.356429]
# zoom = 16
# eo_map = eo_utils.openeoMap(center, zoom)
# eo_map.map
# eo_map.point_coords()

In [8]:
m = leafmap.Map(center=(46.497012, 11.356429), zoom=16)
m

Map(center=[46.497012, 11.356429], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

In [9]:
feat = m.draw_features
geom = feat[0]['geometry']['coordinates']

In [9]:
# x_targ_one = geom[0] + 0.0045
# x_targ_two = geom[0] - 0.0045
# y_targ_one = geom[1] + 0.0045
# y_targ_two = geom[1] - 0.0045
# coords = [(x_targ_one, y_targ_one), 
#           (x_targ_one, y_targ_two),
#           (x_targ_two, y_targ_one),
#           (x_targ_two, y_targ_two)
#          ]
# np.asarray(coords)

In [7]:
geom

[11.354283, 46.500928]

In [10]:
latitude = 40.7128
longitude = -74.0060
distance_km = 1 # Distance in kilometers

# Create a bounding box around the point
bounding_box = create_bounding_box(geom[0], geom[1], distance_km)

# Print the bounding box coordinates
print("Bounding Box (SW Lat, SW Lon, NE Lat, NE Lon):", bounding_box)


Bounding Box (SW Lat, SW Lon, NE Lat, NE Lon): (11.347888783940814, 46.4877501767995, 11.365875216059187, 46.5060958232005)


In [11]:
# geom[0] - 0.045

In [12]:
# bbox = eo_map.getBbox()
# print('west',bbox[0],'\neast',bbox[2],'\nsouth',bbox[1],'\nnorth',bbox[3])

### Recreate process graph

In [5]:
# collection      = 'SENTINEL2_L2A_SENTINELHUB'
# spatial_extent  = {'west':geom[0] - 0.0045,'east':geom[0]+0.0045,'south':geom[1]-0.0045,'north':geom[1]+0.0045,'crs':4326}
# temporal_extent = ["2018-02-01", "2018-02-15"]
# bands           = ['B03', 'B11', 'CLM'] #'SCL']

collection      = 'SENTINEL2_L2A_SENTINELHUB'
# spatial_extent  = {'west':11.020833,'east':11.366667,'south':46.653599,'north':46.954167,'crs':4326}
spatial_extent  = {'west':11.08,'east':11.11,'south':46.77,'north':46.79,'crs':4326}

# temporal_extent = ["2018-02-01", "2018-06-30"]
temporal_extent = ["2018-02-01", "2018-02-15"]
bands           = ['B03', 'B11', 'CLM']

In [6]:
s2cube = conn.load_collection(collection,
                          spatial_extent=spatial_extent,
                          bands=bands,
                          temporal_extent=temporal_extent)

#### Cloud Mask Extraction

In [7]:
cloud_mask= s2cube.band("CLM")
# cloud_mask= s2cube.band("SCL")
# cloud_mask = ( (cloud_mask == 8) | (cloud_mask == 9) | (cloud_mask == 3) ) * 1.0

s2cube_masked = s2cube.mask(cloud_mask)
# s2cube_masked
# cloud_mask = cloud_mask.reduce_dimension(dimension="t", reducer="mean")

#### NDSI Computation

In [12]:
# green = s2cube.band("B03")
# swir = s2cube.band("B11")
green = s2cube_masked.band("B03")
swir = s2cube_masked.band("B11")
ndsi = (green - swir) / (green + swir)

In [13]:
# # ndsi selection
snowmap = (ndsi>0.4) * 1.0
# snowmap.mask(cloud_mask,replacement=2)
# reduce along time
# snowmap  = snowmap.reduce_dimension(dimension="t", reducer="mean")
# snowmap  = snowmap.reduce_temporal(reducer="mean")

# snowmap = snowmap.reduce_bands(reducer="mean")

In [14]:
snowmap = snowmap.save_result(format="GTiff")
snowmap_job = snowmap.create_job(title="Snow Map Computation")
# snowmap_job

In [15]:
snowmap_job.start_and_wait()

0:00:00 Job '0dc9d45d-b6ba-4d23-9442-0dc602046903': send 'start'


OpenEoApiError: [500] Internal: Server error: unsupported operand type(s) for *: 'float' and 'NoneType'

In [ ]:
# import requests
# requests.get()
# ('https://openeo-dev.sinergise.com/testing/jobs', headers={'Authorization': f'Bearer {auth_token}'})

In [40]:
# snowmap_job.delete_job()
# conn.delete("16b8a4a6-72a2-4ee6-8f4e-5cc237070b00")

In [ ]:
snowmap_results = snowmap_job.get_results()
snowmap_results.download_files("data/snowmap_test/")

### Data sharing with STAC
STAC: SpatioTemporal Assets Catalog


In [ ]:
import datetime

import pystac
from pystac.utils import str_to_datetime

import rasterio

# Import extension version
from rio_stac.stac import PROJECTION_EXT_VERSION, RASTER_EXT_VERSION, EO_EXT_VERSION

# Import rio_stac methods
from rio_stac.stac import (
    get_dataset_geom,
    get_projection_info,
    get_raster_info,
    get_eobands_info,
    bbox_to_geom,
)

In [ ]:
assets = [
    {"name": "Snowmap_1", "path": "data/snowmap/1/default.tif", "href": None,"role": None}
    {"name": "Snowmap_2", "path": "data/snowmap/2/default.tif", "href": None,"role": None}
]

media_type = pystac.MediaType.COG #rio_stac.stac.get_media_type
properties = {}
input_datetime = None
id = "snowmap_stac"
collection = "Snowmpa Bolzano"
collection_url = None

extensions =[
    f"https://stac-extensions.github.io/projection/{PROJECTION_EXT_VERSION}/schema.json", 
    f"https://stac-extensions.github.io/raster/{RASTER_EXT_VERSION}/schema.json",
    f"https://stac-extensions.github.io/eo/{EO_EXT_VERSION}/schema.json",
]


In [ ]:
bboxes = []

pystac_assets = []

img_datetimes = []

for asset in assets:
    with rasterio.open(asset["path"]) as src_dst:
        # Get BBOX and Footprint
        dataset_geom = get_dataset_geom(src_dst, densify_pts=0, precision=-1)
        bboxes.append(dataset_geom["bbox"])

        if "start_datetime" not in properties and "end_datetime" not in properties:
            # Try to get datetime from https://gdal.org/user/raster_data_model.html#imagery-domain-remote-sensing
            dst_date = src_dst.get_tag_item("ACQUISITIONDATETIME", "IMAGERY")
            dst_datetime = str_to_datetime(dst_date) if dst_date else None
            if dst_datetime:
                img_datetimes.append(dst_datetime)

        proj_info = {
            f"proj:{name}": value
            for name, value in get_projection_info(src_dst).items()
        }

        raster_info = {
            "raster:bands": get_raster_info(src_dst, max_size=1024)
        }

        eo_info = {}
        eo_info = {"eo:bands": get_eobands_info(src_dst)}
        cloudcover = src_dst.get_tag_item("CLOUDCOVER", "IMAGERY")
        if cloudcover is not None:
            properties.update({"eo:cloud_cover": int(cloudcover)})

        pystac_assets.append(
            (
                asset["name"], 
                pystac.Asset(
                    href=asset["href"] or src_dst.name,
                    media_type=media_type,
                    extra_fields={
                        **proj_info,
                        **raster_info, 
                        **eo_info
                    },
                    roles=asset["role"],
                ),
            )
        )

if img_datetimes and not input_datetime:
    input_datetime = img_datetimes[0]
    
input_datetime = input_datetime or datetime.datetime.utcnow()    

minx, miny, maxx, maxy = zip(*bboxes)
bbox = [min(minx), min(miny), max(maxx), max(maxy)]
            
# item
item = pystac.Item(
    id=id,
    geometry=bbox_to_geom(bbox),
    bbox=bbox,
    collection=collection,
    stac_extensions=extensions,
    datetime=input_datetime,
    properties=properties,
)

# if we add a collection we MUST add a link
if collection:
    item.add_link(
        pystac.Link(
            pystac.RelType.COLLECTION,
            collection_url or collection,
            media_type=pystac.MediaType.JSON,
        )
    )

for key, asset in pystac_assets:
    item.add_asset(key=key, asset=asset)

In [ ]:
item.validate()

In [ ]:
import json

print(json.dumps(item.to_dict(), indent=4))

#### Validate STAC

In [ ]:
#https://github.com/stac-utils/stac-validator
from stac_validator import stac_validator

stac = stac_validator.StacValidate()
stac.validate_dict(json.dumps(item.to_dict(), indent=4))
print(stac.message)